In [21]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama.llms import OllamaLLM
import os
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html#langchain_ollama.llms.OllamaLLM.model

In [22]:
# Load PDF files from a folder
def load_pdfs(folder_path):
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, file))
            documents.extend(loader.load())
    return documents

In [23]:
# Split text into chunks
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=5000, chunk_overlap=50
    )
    return text_splitter.split_documents(documents)

In [24]:
# Initialize Chroma vector database
def setup_chroma(docs, persist_directory="./chroma_db"):
    embeddings = OllamaEmbeddings(model="all-minilm")  # Using model for embeddings
    db = Chroma.from_documents(docs, embeddings, persist_directory=persist_directory)
    return db

In [25]:
# Initialize retrieval and QA system
def get_qa_chain(db):
    retriever = db.as_retriever()
    llm = OllamaLLM(model="llama3.2:1b", num_gpu = -1)


    system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    chain = create_retrieval_chain(retriever, question_answer_chain)
    
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
    return qa_chain

In [26]:
folder_path = "./LocalDocs"  # Change this to your actual folder
documents = load_pdfs(folder_path)
split_docs = split_documents(documents)
db = setup_chroma(split_docs)

In [27]:
embeddings = OllamaEmbeddings(model="all-minilm")  # Using model for embeddings
persist_directory = "./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
qa_chain = get_qa_chain(db)

In [28]:
retriever = db.as_retriever()
docs = retriever.invoke("safety")
docs[0]

Document(id='ba5e2340-d9b5-4b39-9371-a13b2690fdcf', metadata={'page': 101, 'page_label': '102', 'source': './LocalDocs\\MIL-STD-882E.pdf'}, page_content='high, serious, or medium safety risk. \n\uf0b7 A requirement that, if implemented, would negatively impact safety; however \ncode is implemented safely. \n \ne.  Defining and following a process for assessing risk associated with hazards is critical \nto the success of a program, particularly as systems are combined into more complex SoS.  These \nSoS often involve systems developed under disparate development and safety programs and may')

In [29]:
query = "define software safety"
response = qa_chain.invoke(query)
print("AI Response:", response)

AI Response: {'query': 'define software safety', 'result': 'According to the provided context, Software Safety refers to the practice of ensuring that software systems are designed, developed, tested, and used in a way that minimizes the likelihood of failures, hazards, and mishaps. It involves identifying and mitigating the exact software contributors to hazards, as well as increasing confidence that the software will perform as specified to safety and performance requirements while reducing the number of contributors to hazards.'}


In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | f()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

In [ ]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [ ]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)